In [20]:
# ================= Imports =================
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# ================= Load Data =================
train_feature_path = "/Users/jingyuanchen/Desktop/WiDS/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv"
train_label_path = "/Users/jingyuanchen/Desktop/WiDS/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx"
test_feature_path = "/Users/jingyuanchen/Desktop/WiDS/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv"

# 加载特征和标签
df_train = pd.read_csv(train_feature_path)
df_labels = pd.read_excel(train_label_path)
df_test = pd.read_csv(test_feature_path)

# ================= Merge Labels =================
# 合并标签
df = df_train.merge(df_labels, on="participant_id")
X = df.drop(columns=["participant_id", "ADHD_Outcome", "Sex_F"])
y = df[["ADHD_Outcome", "Sex_F"]]
X_test = df_test.drop(columns=["participant_id"])
test_ids = df_test["participant_id"]

# ================= Preprocessing =================
# 缺失值填补 & 标准化
X.fillna(X.median(), inplace=True)
X_test.fillna(X_test.median(), inplace=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# ================= Baseline Model =================
rf = RandomForestClassifier(n_estimators=100, random_state=42)
model = MultiOutputClassifier(rf)
model.fit(X_scaled, y)

# ================= Prediction =================
preds = model.predict(X_test_scaled)

# ================= Submission =================
submission = pd.DataFrame({
    "participant_id": test_ids,
    "ADHD_Outcome": preds[:, 0],
    "Sex_F": preds[:, 1]
})
submission.to_csv("baseline_submission.csv", index=False)
print("✅ Submission saved as baseline_submission.csv")


✅ Submission saved as baseline_submission.csv
